In [1]:
# from two_TrAdaBoostR2 import TwoStageTrAdaBoostR2 ##STrAdaBoost.R2
# from TwoStageTrAdaBoostR2 import TwoStageTrAdaBoostR2 ##two-stage TrAdaBoost.R2

import pandas as pd
import sys
import numpy as np
from pandas import DataFrame
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import RandomForestRegressor


from keras.models import Sequential, load_model, Model
from keras.layers import Input, Dense, Activation, Conv2D, Dropout, Flatten
from keras import optimizers, utils, initializers, regularizers
import keras.backend as K

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler #Importing the StandardScaler

from itertools import combinations

import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats.stats import pearsonr
from math import sqrt

#Geo plotting libraries
import geopandas as gdp
from matplotlib.colors import ListedColormap
# import geoplot as glpt

import xgboost as xgb
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn import linear_model
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
from sklearn.ensemble import GradientBoostingRegressor

from sklearn.model_selection import KFold
import matplotlib.lines as mlines

import statistics

from scipy.stats import rv_continuous
from scipy.stats import *

from statistics import mean
from sklearn.cluster import KMeans
from scipy.spatial import distance


######### Instance Transfer repositories ####################
from adapt.instance_based import TwoStageTrAdaBoostR2

print("Repositories uploaded!!")

Repositories uploaded!!


In [2]:
from adapt.instance_based import TrAdaBoost, TrAdaBoostR2, TwoStageTrAdaBoostR2
from sklearn.model_selection import GridSearchCV
from adapt.instance_based import KMM

print("Second Upload Completed!!")

Second Upload Completed!!


In [3]:
# Importing required Libraries for Plotting the Map

import geopandas as gpd
import json
import altair as alt
import pandas as pd

from os import listdir
from os.path import isfile, join
import glob

import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from matplotlib import cm
import seaborn as sns

In [5]:
# from distributed import Client, LocalCluster

# if __name__ == '__main__':
#     cluster = LocalCluster()
#     client = Client(cluster)
#     print(client)

from dask.distributed import Client

client = Client()

/Users/shrey/anaconda3/lib/python3.8/site-packages/distributed/node.py:151: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 60006 instead
  warnings.warn(


In [7]:
### Reading the US 2018 dataset
US_df = pd.read_csv('US_data/US_2018/US.csv')


In [8]:
from sklearn.model_selection import train_test_split

## Select source and target
US_target = US_df.loc[US_df['Region'] == 'California']
US_source = US_df.loc[US_df['Region'] != 'California']

## Drop unwanted features
drop_features = ['ID', 'Date', 'X', 'Y', 'Lon', 'Lat', 'Region', 'Hour']
US_target = US_target.drop(drop_features, axis = 1)
US_source = US_source.drop(drop_features, axis = 1)

## Split into X and y datasets
target_US = ['PM25']
US_target_y = US_target['PM25']
US_target_X = US_target.drop(target_US, axis = 1)

target_US = ['PM25']
US_source_y = US_source['PM25']
US_source_X = US_source.drop(target_US, axis = 1)


US_train_X, US_test_X, US_train_y, US_test_y = train_test_split(US_target_X, US_target_y, test_size = 0.50, random_state = 42)
US_train_X.shape, US_test_X.shape

((43843, 37), (43843, 37))

In [9]:
import joblib
# from sklearn.externals.joblib import parallel_backend
# top command
# make it print out how much information is being is used.


############## Merging the datasets ##########################################
US_X_df = pd.concat([US_train_X, US_source_X], ignore_index=True)
US_y_df = pd.concat([US_train_y, US_source_y], ignore_index=True)

US_np_train_X = US_X_df.to_numpy()
US_np_train_y = US_y_df.to_numpy()

US_np_test_X = US_test_X.to_numpy()
US_np_test_y = US_test_y.to_numpy()

US_np_train_y_list = US_np_train_y.ravel()
US_np_test_y_list = US_np_test_y.ravel()

src_size_US = len(US_source_y)
tgt_size_US = len(US_train_y)

print("---------------------------")

# src_size_US, tgt_size_US

#################################### STrAdaBoost.R2 Auto ################################################################
from two_TrAdaBoostR2 import TwoStageTrAdaBoostR2

sample_size = [len(US_train_X), len(US_source_X)]
n_estimators = 100
steps = 30
fold = 10
random_state = np.random.RandomState(1)

r2scorelist_stradaboost_auto = []
rmselist_stradaboost_auto = []

for x in range(0, 1):

    model_stradaboost_US = TwoStageTrAdaBoostR2(DecisionTreeRegressor(max_depth = 4),
                        n_estimators = n_estimators, sample_size = sample_size,
                        steps = steps, fold = fold, random_state = random_state)
    
    with joblib.parallel_backend('dask'):
        model_stradaboost_US.fit(US_np_train_X, US_np_train_y_list)
        y_pred_stradaboost_US = model_stradaboost_US.predict(US_np_test_X)


    mse_stradaboost_US = sqrt(mean_squared_error(US_np_test_y, y_pred_stradaboost_US))
    rmselist_stradaboost_US.append(mse_stradaboost_US)

    r2_score_stradaboost_US = pearsonr(US_np_test_y_list, y_pred_stradaboost_US)
    r2_score_stradaboost_US = (r2_score_stradaboost_US[0])**2
    r2scorelist_stradaboost_US.append(r2_score_stradaboost_US)


print("RMSE of STrAdaboostR2:", statistics.mean(rmselist_stradaboost_US))
print("R^2 of STrAdaboostR2:", statistics.mean(r2scorelist_stradaboost_US))
print("\n")
print("RMSE of STrAdaboostR2:", rmselist_stradaboost_US)
print("R^2 of STrAdaboostR2:", r2scorelist_stradaboost_US)


print("-------------------------------------------")

---------------------------
Inside STrAdaBoost.R2


distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker


KeyboardInterrupt: 

distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker


In [7]:
import dask.dataframe as dd

### Reading the US 2018 dataset using Dask
US_df_chunks = dd.from_pandas(US_df, npartitions = 14)
US_df_chunks

,ID,Date,PM25,X,Y,Region,Lon,Lat,Hour,CF_CLDTT,CF_PS,CF_Q,CF_Q10M,CF_Q2M,CF_RH,CF_SLP,CF_T,CF_T10M,CF_T2M,CF_TPREC,CF_TROPPB,CF_TS,CF_U,CF_U10M,CF_U2M,CF_V,CF_V10M,CF_V2M,CF_ZL,CF_ZPBL,LC_Shrubs,LC_Herbaceous,LC_Agriculture,LC_Urban,LC_Bare,LC_Snow,LC_Water,LC_Wetland,LC_Lichen,LC_Closed_Forest,LC_Open_Forest,LC_Ocean,Elevation,Dist_Primary,Dist_Secondary,Pop
npartitions=14,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,int64,object,float64,float64,float64,object,float64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64,float64,int64,float64,float64,float64,int64,float64,float64,float64
49585,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
644605,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
694185,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [ ]:
from dask_ml.model_selection import train_test_split

### Split into features and target 
target_US = ['PM25']
US_target_df = US_df['PM25']
US_features_df = US_df.drop(target_US, axis = 1)

### Drop the unwanted features
drop_features = ['ID', 'Date', 'X', 'Y', 'Region', 'Lon', 'Lat', 'Hour']
US_features_df = US_features_df.drop(drop_features, axis = 1)
US_features_df

In [ ]:
## 10-Fold CV
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

def rmse(score):
    rmse = np.sqrt(-score)
    print(f'rmse= {"{:.2f}".format(rmse)}')

kf = KFold(n_splits = 10, shuffle = True, random_state = 42)

cnt = 1
## split data into training and test set.
for train_idx, test_idx in kf.split(US_features_df, US_target_df):
    print(f'Fold:{cnt}, Train set: {len(train_idx)}, Test set:{len(test_idx)}')
    cnt += 1
    
score_mse_cv = cross_val_score(RandomForestRegressor(random_state = 42, max_depth = 4, n_jobs = 4), US_features_df, US_target_df, cv = kf, scoring = "neg_mean_squared_error")
mean_rmse = rmse(score_mse_cv.mean())
print(f'RMSE scores for each fold are: {score_mse_cv}')
print('Mean rmse: ',mean_rmse)

score_r2_cv = cross_val_score(RandomForestRegressor(random_state= 42), US_features_df, US_target_df, cv = kf, scoring = "r2")
mean_r2 = score_r2_cv.mean()
print(f'R2 scores for each fold are: {score_r2_cv}')
print('Mean r2: ',mean_r2)